<a href="https://colab.research.google.com/github/BrendJ510/Terminal_Economia/blob/main/Optimizacion_RobustadePortafoliosenMercados_Volatiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime

In [ ]:
# 1. Descargar datos (con auto_adjust=True para evitar problemas)
tickers = ["AAPL", "MSFT", "TSLA", "PFE", "KO", "^MXX"]
datos = yf.download(tickers, start="2015-01-01", end="2025-04-04", group_by='ticker', auto_adjust=True)

# 2. Procesar y limpiar los datos
lista_dfs = []
for ticker in tickers:
    try:
        # Extraer datos para cada ticker
        df_ticker = datos[ticker].copy()
        # Agregar columna de identificación
        df_ticker['Acción'] = ticker
        lista_dfs.append(df_ticker)
    except KeyError:
        print(f"No se encontró data para: {ticker}")

# 3. Combinar todos los DataFrames
df_activos = pd.concat(lista_dfs)

# 4. Resetear índice y renombrar columnas
df_activos = df_activos.reset_index().rename(columns={
    'Date': 'Fecha',
    'Open': 'Apertura',
    'High': 'Máximo',
    'Low': 'Mínimo',
    'Close': 'Cierre',  # Cuando auto_adjust=True, Close ya es el precio ajustado
    'Volume': 'Volumen'
})

# 5. Seleccionar solo las columnas existentes
columnas_disponibles = ['Fecha', 'Acción', 'Apertura', 'Máximo', 'Mínimo', 'Cierre', 'Volumen']
df_activos = df_activos[columnas_disponibles]

# 6. Ordenar por fecha y acción
df_activos= df_activos.dropna()
df_activos = df_activos.sort_values(['Fecha', 'Acción'])
df_activos.columns.name = None

# Usar la columna 'Fecha' como nuevo índice
df_activos = df_activos.set_index('Fecha')
# Verificar resultado
print("\nDataFrame completamente limpio:")
print(df_activos.head())

[*********************100%***********************]  6 of 6 completed


DataFrame completamente limpio:
           Acción   Apertura     Máximo     Mínimo     Cierre      Volumen
Fecha                                                                     
2015-01-02   AAPL  24.778679  24.789802  23.879981  24.320433  212818400.0
2015-01-02     KO  30.524182  30.625306  30.191927  30.437508    9921100.0
2015-01-02   MSFT  39.986432  40.637731  39.883596  40.072128   27913900.0
2015-01-02    PFE  19.677466  19.998396  19.646001  19.715221   16371571.0
2015-01-02   TSLA  14.858000  14.883333  14.217333  14.620667   71466000.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_activos.to_excel('/content/drive/MyDrive/Portafolio.xlsx')

In [ ]:
df_activos = pd.read_excel('/content/drive/MyDrive/Portafolio.xlsx')
df_activos.head()

,Fecha,Acción,Apertura,Máximo,Mínimo,Cierre,Volumen
0,2015-01-02,AAPL,24.778679,24.789802,23.879981,24.320433,212818400
1,2015-01-02,KO,30.524182,30.625306,30.191927,30.437508,9921100
2,2015-01-02,MSFT,39.986432,40.637731,39.883596,40.072128,27913900
3,2015-01-02,PFE,19.677466,19.998396,19.646001,19.715221,16371571
4,2015-01-02,TSLA,14.858000,14.883333,14.217333,14.620667,71466000


In [ ]:
# 1. Retorno diario (%)
df_activos['Retorno Diario'] = df_activos.groupby('Acción')['Cierre'].pct_change() * 100
df_activos[['Cierre','Retorno Diario']]
# 2. Volatilidad (Desviación Estándar de rendimientos, ventana móvil 20 días)
df_activos['Volatilidad_50d'] = df_activos.groupby('Acción')['Retorno Diario'].rolling(window= 50).std().reset_index( drop=True)
df_activos[['Cierre','Retorno Diario','Volatilidad_50d']].head(100)
# Volatilidad anualizada (usando Cierre)
df_activos['Volatilidad_50d_anual'] = df_activos['Volatilidad_50d'] * np.sqrt(252)  # 252 días hábiles/año


In [ ]:
df_activos[['Cierre','Retorno Diario','Volatilidad_50d','Volatilidad_50d_anual']]

,Cierre,Retorno Diario,Volatilidad_50d,Volatilidad_50d_anual
0,24.320433,NaN,NaN,NaN
1,30.437508,NaN,NaN,NaN
2,40.072128,NaN,NaN,NaN
3,19.715221,NaN,NaN,NaN
4,14.620667,NaN,NaN,NaN
...,...,...,...,...
15469,73.180000,2.593577,0.824359,13.086294
15470,373.109985,-2.363016,0.842578,13.375514
15471,24.290001,-1.659918,0.869443,13.801974
15472,267.279999,-5.474611,0.873954,13.873591


In [ ]:
# 1. Calcular pico histórico y drawdown actual
df_activos['Pico'] = df_activos.groupby('Acción')['Cierre'].cummax()
df_activos['Drawdown_%'] = (df_activos['Cierre'] - df_activos['Pico']) / df_activos['Pico'] * 100

# 2. Calcular el máximo drawdown por acción (resumen)
max_drawdown = df_activos.groupby('Acción')['Drawdown_%'].min().reset_index()
max_drawdown.columns = ['Acción', 'Max_Drawdown_%']

# 3. Opcional: Añadir columna que indique si es el peor día (para gráficos)
df_activos['Es_Max_Drawdown'] = df_activos.groupby('Acción')['Drawdown_%'].transform(lambda x: x == x.min())

# Ver resultados
print("\n Máximos Drawdowns por Activo:")
print(max_drawdown)

print("\n Peores días históricos:")
print(df_activos[df_activos['Es_Max_Drawdown'] == True][['Acción', 'Fecha', 'Drawdown_%']])


 Máximos Drawdowns por Activo:
  Acción  Max_Drawdown_%
0   AAPL      -38.515928
1     KO      -36.987520
2   MSFT      -37.148483
3    PFE      -54.818030
4   TSLA      -73.632217
5   ^MXX      -36.255918

 Peores días históricos:
      Acción      Fecha  Drawdown_%
6041    AAPL 2019-01-03  -38.515928
7877      KO 2020-03-23  -36.987520
7881    ^MXX 2020-03-23  -36.255918
11845   MSFT 2022-11-03  -37.148483
12088   TSLA 2023-01-03  -73.632217
14060    PFE 2024-04-25  -54.818030


In [ ]:
# 1. Calcular rango diario (correcto)
df_activos['Rango_Diario'] = (df_activos['Máximo'] - df_activos['Mínimo']) / df_activos['Apertura'] * 100

# 2. Calcular el promedio por acción (PERO guardarlo en una variable separada)
rango_promedio = df_activos.groupby('Acción')['Rango_Diario'].mean().reset_index()
rango_promedio.columns = ['Acción', 'Rango_Promedio_%']

# 3. Mostrar resultados relevantes
print("\n Rango Diario Promedio por Acción (%):")
print(rango_promedio)

# 4.  Ver algunos datos diarios )
print("\n Muestra de datos diarios:")
print(df_activos[['Acción', 'Fecha', 'Rango_Diario']].head())


 Rango Diario Promedio por Acción (%):
  Acción  Rango_Promedio_%
0   AAPL          2.008165
1     KO          1.363762
2   MSFT          1.912742
3    PFE          1.867039
4   TSLA          4.134861
5   ^MXX          1.297627

 Muestra de datos diarios:
  Acción      Fecha  Rango_Diario
0   AAPL 2015-01-02      3.671787
1     KO 2015-01-02      1.419788
2   MSFT 2015-01-02      1.885978
3    PFE 2015-01-02      1.790853
4   TSLA 2015-01-02      4.482436


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_activos.to_excel('/content/drive/MyDrive/Portafolio2.xlsx', index=False)


In [ ]:
df_activos = pd.read_excel('/content/drive/MyDrive/Portafolio2.xlsx')
df_activos.head(2000)

,Fecha,Acción,Apertura,Máximo,Mínimo,Cierre,Volumen,Retorno Diario,Volatilidad_50d,Volatilidad_50d_anual,Pico,Drawdown_%,Es_Max_Drawdown,Rango_Diario
0,2015-01-02,AAPL,24.778679,24.789802,23.879981,24.320433,212818400,NaN,NaN,NaN,24.320433,0.000000,False,3.671787
1,2015-01-02,KO,30.524182,30.625306,30.191927,30.437508,9921100,NaN,NaN,NaN,30.437508,0.000000,False,1.419788
2,2015-01-02,MSFT,39.986432,40.637731,39.883596,40.072128,27913900,NaN,NaN,NaN,40.072128,0.000000,False,1.885978
3,2015-01-02,PFE,19.677466,19.998396,19.646001,19.715221,16371571,NaN,NaN,NaN,19.715221,0.000000,False,1.790853
4,2015-01-02,TSLA,14.858000,14.883333,14.217333,14.620667,71466000,NaN,NaN,NaN,14.620667,0.000000,False,4.482436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2016-04-28,^MXX,45940.980469,45982.839844,45189.628906,45528.929688,378055500,-0.898076,2.727111,43.291549,46191.511719,-1.434424,False,1.726587
1996,2016-04-29,AAPL,21.379676,21.545728,21.043025,21.322809,274126000,-1.149453,2.750087,43.656284,29.723492,-28.262771,False,2.351313
1997,2016-04-29,KO,33.493641,33.741855,33.471077,33.696724,11716000,0.380972,2.754213,43.721773,35.268719,-4.457193,False,0.808445
1998,2016-04-29,MSFT,43.754943,44.552907,43.754943,44.215988,48411700,-0.060123,2.755263,43.738447,50.058853,-11.671991,False,1.823711


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear figura con eje Y secundario
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Colores para cada acción
colores = {
    'AAPL': 'cyan',
    'TSLA': 'orange',
    'KO': 'lightgreen',
    'MSFT': 'violet',
    'PFE': 'deeppink',
    '^MXX': 'darkcyan'
}

# Añadir todas las acciones (excepto ^MXX) al eje Y principal
for accion in ['AAPL', 'TSLA', 'KO', 'MSFT', 'PFE']:
    df_accion = df_activos[df_activos['Acción'] == accion]
    fig.add_trace(go.Scatter(
        x=df_accion['Fecha'],
        y=df_accion['Cierre'],
        mode='lines',
        name=accion,
        line=dict(color=colores[accion], width=2)
    ), secondary_y=False)

# Añadir ^MXX al eje Y secundario
df_mxx = df_activos[df_activos['Acción'] == '^MXX']
fig.add_trace(go.Scatter(
    x=df_mxx['Fecha'],
    y=df_mxx['Cierre'],
    mode='lines',
    name='^MXX',
    line=dict(color=colores['^MXX'], width=2, dash='dot')
), secondary_y=True)

# Configuración del layout
fig.update_layout(
    title='Precios de Cierre (Acciones vs Índice)',
    xaxis_title='Fecha',
    template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.1, xanchor='right', x=1),
    hovermode='x unified'
)

# Configurar ejes Y
fig.update_yaxes(title_text='Precio Acciones (USD)', secondary_y=False)
fig.update_yaxes(title_text='Índice ^MXX', secondary_y=True)

# Rangeselector (conserva tu configuración original)
fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label='1M', step='month', stepmode='backward'),
            dict(count=6, label='6M', step='month', stepmode='backward'),
            dict(count=1, label='1Y', step='year', stepmode='backward'),
            dict(step='all', label='Todo')
        ])
    ),
    rangeslider=dict(visible=True),
    type='date'
)

fig.show()

In [ ]:
import plotly.express as px

# 1. Preparar los datos
df_boxplot = df_activos.reset_index()[['Acción', 'Retorno Diario']]

# 2. Crear el boxplot con cajas claras y bigotes definidos
fig = px.box(
    df_boxplot,
    x='Acción',
    y='Retorno Diario',
    color='Acción',
    points=False,  # Oculta puntos individuales (para versión clásica)
    title="Distribución de Rendimientos Diarios (Boxplot Clásico Interactivo)",
    labels={'Retorno Diario': 'Rendimiento Diario (%)'},
    template='plotly_white',
    color_discrete_map={  # Colores personalizados para las cajas
        'AAPL': 'rgba(55, 128, 191, 0.7)',
        'TSLA': 'rgba(219, 64, 82, 0.7)',
        'KO': 'rgba(50, 171, 96, 0.7)'
    }
)

# 3. Personalizar estilo para que parezca un boxplot "de libro"
fig.update_traces(
    boxmean=True,  # Muestra la media como línea punteada
    whiskerwidth=0.2,  # Grosor de los bigotes
    marker=dict(opacity=0),  # Oculta marcadores (outliers se verán después)
    line=dict(width=2, color='black')  # Bordes negros para las cajas
)

import plotly.express as px
import plotly.graph_objects as go

# 1. Preparar los datos
df_boxplot = df_activos.reset_index()[['Acción', 'Retorno Diario']]

# 2. Crear el boxplot con cajas claras y bigotes definidos
fig = px.box(
    df_boxplot,
    x='Acción',
    y='Retorno Diario',
    color='Acción',
    points=False,  # Oculta puntos individuales
    title="Distribución de Rendimientos Diarios (Boxplot Clásico Interactivo)",
    labels={'Retorno Diario': 'Rendimiento Diario (%)'},
    template='plotly_white',
    color_discrete_map={  # Colores personalizados para las cajas
        'AAPL': 'rgba(55, 128, 191, 0.7)',
        'TSLA': 'rgba(219, 64, 82, 0.7)',
        'KO': 'rgba(50, 171, 96, 0.7)'
    }
)

# 3. Personalizar estilo para que parezca un boxplot
fig.update_traces(
    boxmean=True,  # Muestra la media como línea punteada
    whiskerwidth=0.2,  # Grosor de los bigotes
    marker=dict(opacity=0),  # Oculta marcadores
    line=dict(width=2, color='black')  # Bordes negros para las cajas
)

outliers = df_boxplot.groupby('Acción').apply(
    lambda x: x[
        (x['Retorno Diario'] > (x['Retorno Diario'].quantile(0.75) + 1.5*(x['Retorno Diario'].quantile(0.75) - x['Retorno Diario'].quantile(0.25)))) |
        (x['Retorno Diario'] < (x['Retorno Diario'].quantile(0.25) - 1.5*(x['Retorno Diario'].quantile(0.75) - x['Retorno Diario'].quantile(0.25))))
    ]).reset_index(drop=True)

fig.add_trace(go.Scatter(
    x=outliers['Acción'],
    y=outliers['Retorno Diario'],
    mode='markers',
    marker=dict(color='red', size=5),
    name='Outliers',
    hoverinfo='y+name'
))

# 5. Añadir línea en 0% y ajustar layout
fig.add_hline(y=0, line_dash="dot", line_color="gray")
fig.update_layout(
    showlegend=False,
    yaxis_title='Rendimiento Diario (%)',
    xaxis_title='',
    hovermode='x'
)


<ipython-input-32-4d6f93676659>:62: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [ ]:
# Calcular volumen promedio por acción
vol_promedio = df_activos.groupby('Acción')['Volumen'].mean().reset_index()

# Crear gráfico de barras
fig_barras = go.Figure(go.Bar(
    x=vol_promedio['Acción'],
    y=vol_promedio['Volumen'],
    marker_color='rgba(55, 128, 191, 0.7)',  # Mismo azul que en el boxplot
    text=vol_promedio['Volumen'].round(0),
    textposition='auto'
))

fig_barras.update_layout(
    title="Volumen Promedio de Negociación por Acción",
    xaxis_title="Acción",
    yaxis_title="Volumen (acciones)",
    template='plotly_white',
    hovermode='x',
    showlegend=False,
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

# Añadir línea horizontal de referencia
fig_barras.add_hline(
    y=vol_promedio['Volumen'].mean(),
    line_dash="dot",
    line_color="red",
    annotation_text="Media global"
)

fig_barras.show()

In [ ]:
import plotly.graph_objects as go

# Calcular matriz de correlación
corr_matrix = df_activos.pivot_table(index='Fecha', columns='Acción', values='Retorno Diario').corr()

# Crear mapa de calor
fig_heatmap = go.Figure(go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.index,
    colorscale='RdBu',  # Esquema rojo-azul (similar a Seaborn)
    zmin=-1,
    zmax=1,
    hoverongaps=False,
    text=corr_matrix.round(2).values,  # Mostrar valores con 2 decimales
    texttemplate="%{text}",
))

fig_heatmap.update_layout(
    title="Matriz de Correlación entre Activos",
    xaxis_title="",
    yaxis_title="",
    template='plotly_white',
    height=600,
    xaxis=dict(tickangle=-45),
    coloraxis_colorbar=dict(title="Correlación")
)

fig_heatmap.update_traces(
    textfont_size=12,
    textfont_color="white"  # Texto blanco para mejor contraste
)

fig_heatmap.show()